#  WEBSCRAPPING autoscout24

- Importe de librerías necesarias

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

In [2]:
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=c_options)

In [3]:
# Abrir una página web
url = "https://www.autoscout24.es/lst/tesla?atype=C&cy=D%2CA%2CB%2CE%2CF%2CI%2CL%2CNL&damaged_listing=exclude&desc=0&powertype=kw&search_id=22r4ql331ip&sort=standard&source=homepage_search-mask&ustate=N%2CU"
driver.get(url)

try:
    wait = WebDriverWait(driver, 10)
    cookie_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "_consent-accept_1lphq_114")))

    cookie_button.click()
    print("Cookies aceptadas correctamente.")

    time.sleep(5)
    

except TimeoutException:
    print("Error: El botón de aceptar cookies no apareció a tiempo.")
except NoSuchElementException:
    print("Error: No se encontró el botón de aceptar cookies.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")
finally:
    driver.quit()

Cookies aceptadas correctamente.
